In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import joblib

In [3]:
import dask.dataframe as dd
from dask.distributed import Client

In [6]:
client = Client(n_workers=12, memory_limit='16GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 178.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59206,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 178.81 GiB
Comm: tcp://127.0.0.1:59290,Total threads: 1
Dashboard: http://127.0.0.1:59291/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:59209,


In [20]:
client.close()

In [1]:
import joblib

# Load the PCA-transformed data
df_pca_all = joblib.load('pca_transformed_data.pkl')


In [3]:
# Import the necessary library for Isolation Forest
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Assuming df_pca_all is already defined and is the result of PCA
# Scale the PCA-transformed data
scaler = StandardScaler()
df_pca_scaled = scaler.fit_transform(df_pca_all)

# Define the contamination value (adjust based on your dataset)
contamination_value = 0.01  # Adjust this value according to the expected number of anomalies

# Initialize and fit the Isolation Forest model with scaled data
isolation_forest = IsolationForest(contamination=contamination_value, random_state=42)
labels = isolation_forest.fit_predict(df_pca_scaled)

# Print the new number of anomalies (label == -1 indicates an anomaly)
print(f"Number of detected anomalies (potential attacks): {sum(labels == -1)}")


Number of detected anomalies (potential attacks): 95369


In [2]:
# Apply Anomaly Detection on Loaded PCA Data# Option 1: Isolation Forest
# Option 1: Isolation Forest
from sklearn.ensemble import IsolationForest

# Fit the Isolation Forest model to the loaded PCA-transformed data
isolation_forest = IsolationForest(contamination='auto', random_state=42)
labels = isolation_forest.fit_predict(df_pca_all)

# Isolation Forest assigns -1 to anomalies (potential attacks) and 1 to normal points
# Print the number of anomalies
print(f"Number of detected anomalies (potential attacks): {sum(labels == -1)}")

Number of detected anomalies (potential attacks): 657627


In [3]:
## 1. Adjust the Contamination Parameter
# Calculate the contamination proportion based on the expected anomalies
expected_anomalies = 330
total_samples = len(df_pca_all)
contamination_value = expected_anomalies / total_samples

# Refit the Isolation Forest with adjusted contamination
isolation_forest = IsolationForest(contamination=contamination_value, random_state=42)
labels = isolation_forest.fit_predict(df_pca_all)

# Print the new number of anomalies
print(f"Number of detected anomalies (potential attacks): {sum(labels == -1)}")


Number of detected anomalies (potential attacks): 330


In [ ]:
## 2. Use Local Outlier Factor (LOF)

from sklearn.neighbors import LocalOutlierFactor

# Fit the LOF model to the PCA-transformed data
lof = LocalOutlierFactor(n_neighbors=20, contamination=contamination_value)
labels_lof = lof.fit_predict(df_pca_all)

# LOF assigns -1 to anomalies and 1 to normal points
print(f"Number of detected anomalies (potential attacks) with LOF: {sum(labels_lof == -1)}")


In [ ]:
## 3. Visualize and Analyze Anomalies

import matplotlib.pyplot as plt

# Get the anomaly scores from the Isolation Forest
anomaly_scores = isolation_forest.decision_function(df_pca_all)

# Plot the distribution of anomaly scores
plt.hist(anomaly_scores, bins=50)
plt.title('Anomaly Score Distribution')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.show()


In [24]:
# If you want to store the labels in a DataFrame to examine them alongside the original data:
import pandas as pd

# Create a DataFrame with the PCA-transformed data and labels
df_pca_results = pd.DataFrame(df_pca_all, columns=[f'PC{i+1}' for i in range(df_pca_all.shape[1])])
df_pca_results['anomaly_label'] = labels

# Save the results to a file for further analysis if needed
df_pca_results.to_csv('pca_anomaly_results.csv', index=False)

KeyboardInterrupt: 

In [22]:
# Plot the anomalies in the first two principal components for visualization
import matplotlib.pyplot as plt

plt.scatter(df_pca_all[:, 0], df_pca_all[:, 1], c=labels, cmap='coolwarm')
plt.title("Anomaly Detection with Isolation Forest (First 2 PCA Components)")
plt.xlabel("First Principal Component")
plt.ylabel("Second Principal Component")
plt.colorbar(label="Anomaly Label")
plt.show()



Number of detected anomalies (potential attacks): 657627


KeyboardInterrupt: 

In [ ]:
# One Class SVM 

from sklearn.svm import OneClassSVM

# Fit the One-Class SVM model to the loaded PCA-transformed data
svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)
labels_svm = svm.fit_predict(df_pca_all)

# One-Class SVM assigns -1 to anomalies (potential attacks) and 1 to normal points
print(f"Number of detected anomalies (potential attacks): {sum(labels_svm == -1)}")

# Create a DataFrame with the PCA-transformed data and labels
df_pca_results_svm = pd.DataFrame(df_pca_all, columns=[f'PC{i+1}' for i in range(df_pca_all.shape[1])])
df_pca_results_svm['svm_anomaly_label'] = labels_svm

# Save the results to a file for further analysis
df_pca_results_svm.to_csv('pca_svm_anomaly_results.csv', index=False)



In [ ]:
# Plot the anomalies in the first two principal components for visualization
plt.scatter(df_pca_all[:, 0], df_pca_all[:, 1], c=labels_svm, cmap='coolwarm')
plt.title("Anomaly Detection with One-Class SVM (First 2 PCA Components)")
plt.xlabel("First Principal Component")
plt.ylabel("Second Principal Component")
plt.colorbar(label="Anomaly Label")
plt.show()


In [9]:
df_dropped_phase2 = dd.read_csv(r'df_dropped_phase2.csv', blocksize = '80MB')
df_dropped_phase2
df_dropped_phase2.head()

,ts,flow_duration,Header_Length,Source Port,Destination Port,Protocol Type,Duration,Rate,Srate,Drate,...,Covariance,Variance,Weight,DS status,Fragments,Sequence number,Protocol Version,flow_idle_time,flow_active_time,label
0,1.701728e+09,0.000000,1514,80,48884,6,127,0.000000,0.000000,0.0,...,0.000000,0.0,1,0,0,0,0,1.701728e+09,0.000000,0
1,1.701728e+09,0.000548,1568,48884,80,6,64,3650.395126,3650.395126,0.0,...,532900.000000,1.0,4,0,0,0,0,5.478859e-04,0.000548,0
2,1.701728e+09,0.002528,1622,48884,80,6,64,1186.731302,1186.731302,0.0,...,473688.888889,1.0,9,0,0,0,0,1.980066e-03,0.002528,0
3,1.701728e+09,0.004881,1676,48884,80,6,64,819.520125,819.520125,0.0,...,399675.000000,1.0,16,0,0,0,0,2.352953e-03,0.004881,0
4,1.701728e+09,0.007555,3189,80,48884,6,127,661.812674,661.812674,0.0,...,511233.760000,1.0,25,0,0,0,0,2.674103e-03,0.007555,0


In [15]:
# Remove leading and trailing spaces from column names
df_dropped_phase2.columns = df_dropped_phase2.columns.str.strip()  

In [14]:
# Convert Dask DataFrame to Pandas
df_dropped_computed = df_dropped_phase2.compute()  

In [16]:
# Standardize the data (mean = 0, variance = 1)
scaler = StandardScaler()
df_standardized = scaler.fit_transform(df_dropped_computed)

In [ ]:


# Applying PCA
pca = PCA()
pca.fit(df_standardized)
df_pca_all = pca.transform(df_standardized)

# Eigenvalues and variance explained
eigenvalues = pca.explained_variance_
explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot eigenvalues (scree plot)
plt.bar(np.arange(0, df_standardized.shape[1], 1), eigenvalues)
plt.plot(eigenvalues, "r")
plt.plot(eigenvalues, "ro")
plt.xlabel("Principal Components")
plt.ylabel("Eigenvalues")
plt.title("Scree Plot of Eigenvalues")
plt.show()

# Plot the explained variance ratio
plt.plot(np.arange(1, len(explained_variance_ratio) + 1), explained_variance_ratio)
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("Explained Variance vs Number of Components")
plt.axhline(y=0.90, color='r', linestyle='--', label='90% Variance')
plt.legend()
plt.show()

# Print the number of components explaining 90% of variance
n_components_90 = np.argmax(explained_variance_ratio >= 0.90) + 1
print(f"Number of components explaining 90% of variance: {n_components_90}")

# Save the PCA model and the transformed data
joblib.dump(pca, 'pca_model.pkl')
joblib.dump(df_pca_all, 'pca_transformed_data.pkl')
joblib.dump(n_components_90, 'n_components_90.pkl')
